In [1]:
import pandas as pd

In [2]:
data_dict = pd.read_table("2025_Problem_C_Data\\data_dictionary.csv", encoding="Windows-1252",sep=",")

# 变量说明

In [3]:
data_dict.iloc[0:8,]

,summerOly_medal_counts.csv,Unnamed: 1,Unnamed: 2
0,variables,explanation,example
1,Rank,Rank of country based on total medals won,"1, 2"
2,NOC,Name of country as recorded for that Olympics,"China, France"
3,Gold,Number of Gold medals the country earned,"0, 1, 2"
4,Silver,Number of Silver medals the country earned,"0, 1, 2"
5,Bronze,Number of Bronze medals the country earned,"0, 1, 2"
6,Total,Number of Total medals the country earned,"0, 1, 2"
7,Year,Olympic year,"1896, 1900"


In [4]:
data_dict.iloc[11:14,]

,summerOly_medal_counts.csv,Unnamed: 1,Unnamed: 2
11,variables,explanation,example
12,Year,Olympic year,"1896, 1900"
13,Host,"Location where olympics were held (city, country)","St. Louis, United States"


In [5]:
data_dict.iloc[17:36,]

,summerOly_medal_counts.csv,Unnamed: 1,Unnamed: 2
17,variables,explanation,example
18,Sport,Sport category,"Aquatics, Cycling"
19,Discipline,Discipline within the sport category,"Artistic swimming, Diving"
20,Code,Code for the discipline,"SWA, DIV"
21,Year,Number of events in disipline for that year,"0, 1"
22,Data set of current and discontinued summer pr...,NaN,NaN
23,Column Descriptors,NaN,NaN
24,Sport and Discipline:,The sports (and disciplines) make up the curre...,NaN
25,Code:,Each discipline is marked with a unique 3-char...,NaN
26,Sports Governing Body:,A listing of the sports governing bodies assoc...,NaN


In [6]:
data_dict.iloc[39:50,]

,summerOly_medal_counts.csv,Unnamed: 1,Unnamed: 2
39,variables,explanation,example
40,Name,Athlete name,"Cai Yun, Carl Lewis"
41,Sex,Sex of athlete,"F, M"
42,Team,Country name as recorded at the Olympics,"China, France"
43,NOC,Name of Country (code),"CHN, FRA"
44,Year,Olympic year,"1896, 1900"
45,Season,Type of Olympics,Summer
46,City,City where Olympics were held,"Beijing, Paris"
47,Sport,Sport category,"Aquatics, Cycling"
48,Event,Event within the sport,Gymnastics Men's Rings


以下是对该文件内容的翻译：

---

### **文件 1：夏季奥运会奖牌统计 (summerOly_medal_counts.csv)**

| 变量名称   | 解释            | 示例           |
|--------|---------------|--------------|
| Rank   | 国家奖牌总数排名      | "1, 2"       |
| NOC    | 该奥运会记录的国家名称   | "中国, 法国"     |
| Gold   | 国家获得的金牌数量     | "0, 1, 2"    |
| Silver | 国家获得的银牌数量     | "0, 1, 2"    |
| Bronze | 国家获得的铜牌数量     | "0, 1, 2"    |
| Total  | 国家获得的奖牌总数     | "0, 1, 2"  |
| Year   | 奥运会年份         | "1896, 1900" |

---

### **文件 2：夏季奥运会举办地 (summerOly_hosts.csv)**

| 变量名称 | 解释                                       | 示例                      |
|----------|--------------------------------------------|---------------------------|
| Year     | 奥运会年份                                 | "1896, 1900"              |
| Host     | 奥运会举办地（城市，国家）                 | "圣路易斯, 美国"          |

---

### **文件 3：夏季奥运会项目 (summerOly_programs.csv)**

| 变量名称       | 解释              | 示例          |
|------------|-----------------|-------------|
| Sport      | 体育类别            | "水上运动, 自行车" |
| Discipline | 体育类别内的具体项目      | "艺术体操, 跳水"  |
| Code       | 项目的代码           | "SWA, DIV"  |
| Year       | 该项目在某年奥运会中的比赛数量 | "0, 1"      |

#### 数据说明：
- **体育和项目 (Sport and Discipline)**：包括当前和已取消的夏季奥运会官方项目，按国际奥委会（IOC）使用的名称按字母顺序排列。
- **代码 (Code)**：每个项目由 IOC 分配一个独特的 3 字符标识符代码。
- **体育管理机构**：列出与奥运会体育相关的国际管理机构。
- **年份 (1896-2028)**：列的四位数代表相关奥运会的年份。
- 表格中的数字表示该项目在该年举办的比赛数量；某些项目可能标记为展示项目（非正式项目）。

---

### **文件 4：夏季奥运会运动员数据 (summerOly_athletes.csv)**

| 变量名称 | 解释                                   | 示例                       |
|----------|----------------------------------------|----------------------------|
| Name     | 运动员姓名                             | "蔡赟, 卡尔·刘易斯"        |
| Sex      | 运动员性别                             | "女 (F), 男 (M)"           |
| Team     | 国家名称（奥运会记录的）               | "中国, 法国"               |
| NOC      | 国家代码                               | "CHN, FRA"                 |
| Year     | 奥运会年份                             | "1896, 1900"               |
| Season   | 奥运会类型                             | 夏季 (Summer)              |
| City     | 奥运会举办城市                         | "北京, 巴黎"               |
| Sport    | 体育类别                               | "水上运动, 自行车"         |
| Event    | 项目名称                               | "体操男子吊环"             |
| Medal    | 获得的奖牌类型                         | "无奖牌, 金牌 (Gold)"      |

---

### **备注**
1. **1906年奥运会**：原计划每四年举办一次，地位与正式奥运会相同，但仅在 1906 年于雅典举办了一次，之后取消。
2. **冬季项目迁移**：在 1924 年之前，一些冰上项目（如花样滑冰和冰球）在夏季奥运会中举办，1924 年起转移至冬奥会。
3. **管理机构变化**：如拳击的管理机构从 2023 年起由 AIBA 改为 IBA。

---

# 获取分析历史奖牌数据

In [7]:
history_medal_counts = pd.read_csv("2025_Problem_C_Data\\summerOly_medal_counts.csv", encoding="Windows-1252",sep=",")

In [8]:
history_medal_counts

,Rank,NOC,Gold,Silver,Bronze,Total,Year
0,1,United States,11,7,2,20,1896
1,2,Greece,10,18,19,47,1896
2,3,Germany,6,5,2,13,1896
3,4,France,5,4,2,11,1896
4,5,Great Britain,2,3,2,7,1896
...,...,...,...,...,...,...,...
1430,84,Qatar,0,0,1,1,2024
1431,84,Refugee Olympic Team,0,0,1,1,2024
1432,84,Singapore,0,0,1,1,2024
1433,84,Slovakia,0,0,1,1,2024


# 对于历史上已经消失或者合并的国家的数据进行清洗

In [9]:
# 获取国家的名称
noc_names = history_medal_counts["NOC"].unique()
print(noc_names)

['United States' 'Greece' 'Germany' 'France' 'Great Britain' 'Hungary'
 'Austria' 'Australia' 'Denmark' 'Switzerland' 'Mixed team' 'Belgium'
 'Italy' 'Cuba' 'Canada' 'Spain' 'Luxembourg' 'Norway' 'Netherlands'
 'India' 'Bohemia' 'Sweden' 'Australasia' 'Russian Empire' 'Finland'
 'South Africa' 'Estonia' 'Brazil' 'Japan' 'Czechoslovakia' 'New Zealand'
 'Yugoslavia' 'Argentina' 'Uruguay' 'Poland' 'Haiti' 'Portugal' 'Romania'
 'Egypt' 'Ireland' 'Chile' 'Philippines' 'Mexico' 'Latvia' 'Turkey'
 'Jamaica' 'Peru' 'Ceylon' 'Trinidad and Tobago' 'Panama' 'South Korea'
 'Iran' 'Puerto Rico' 'Soviet Union' 'Lebanon' 'Bulgaria' 'Venezuela'
 'United Team of Germany' 'Iceland' 'Pakistan' 'Bahamas' 'Ethiopia'
 'Formosa' 'Ghana' 'Morocco' 'Singapore' 'British West Indies' 'Iraq'
 'Tunisia' 'Kenya' 'Nigeria' 'East Germany' 'West Germany' 'Mongolia'
 'Uganda' 'Cameroon' 'Taiwan' 'North Korea' 'Colombia' 'Niger' 'Bermuda'
 'Thailand' 'Zimbabwe' 'Tanzania' 'Guyana' 'China' 'Ivory Coast' 'Syria'
 'Algeria

In [10]:
# Step 1: Redefine the removal and merge rules based on updated requirements
# Countries/regions to remove entirely
final_remove_nocs = [
    "Bohemia", "Australasia", "Russian Empire", "Czechoslovakia",
    "Unified Team", "Mixed team", "East Germany", "West Germany",
    "United Team of Germany", "FR Yugoslavia", "Serbia and Montenegro",
    "Soviet Union"
]

# Update merge mappings
final_merge_nocs = {
    "Formosa": "Chinese Taipei",
    "Hong Kong":"Hong Kong, China",
    "AIN":"Russia"
    # Merge Formosa with Chinese Taipei
    # Note: China, Hong Kong, and Macau remain separate
}

# Step 2: Start cleaning process from original data
final_cleaned_data = history_medal_counts[~history_medal_counts["NOC"].isin(final_remove_nocs)]

# Replace NOC names based on updated merge mapping
final_cleaned_data["NOC"] = final_cleaned_data["NOC"].replace(final_merge_nocs)

# Step 3: Recalculate totals and ranks with the updated cleaning rules
final_cleaned_data = (
    final_cleaned_data.groupby(["Year", "NOC"], as_index=False)
    .agg({"Gold": "sum", "Silver": "sum", "Bronze": "sum"})
)
final_cleaned_data["Total"] = final_cleaned_data["Gold"] + final_cleaned_data["Silver"] + final_cleaned_data["Bronze"]

# Rank NOCs within each year
final_cleaned_data["Rank"] = (
    final_cleaned_data.sort_values(["Year", "Total", "Gold", "Silver"], ascending=[True, False, False, False])
    .groupby("Year")
    .cumcount() + 1
)

# 保存数据
final_cleaned_data.head()
cleaned_medal_data = final_cleaned_data
cleaned_medal_data.to_csv("2025_Problem_C_Data_cleaned\\cleaned_medal_data.csv", index=False)


C:\Users\Tieti_Tage\AppData\Local\Temp\ipykernel_18968\2345873581.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_cleaned_data["NOC"] = final_cleaned_data["NOC"].replace(final_merge_nocs)


In [11]:
# # 遍历分组
# for name, group in final_cleaned_data.groupby("NOC"):
#     # 绘制每个 NOC 的奖牌柱状图
#     group.plot(x="Year", y=["Gold", "Silver", "Bronze"], kind="bar", title=name)
#     plt.tight_layout()  # 调整布局
#     plt.show()  # 显示每个图


- 清洗后得到的各个国家和地区的历史奖牌的柱状图如图

# 读取项目表

In [12]:
program_data = pd.read_csv("2025_Problem_C_Data\\summerOly_programs.csv", encoding="latin1",sep=",")

In [13]:
program_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 35 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Sport                  74 non-null     object 
 1   Discipline             72 non-null     object 
 2   Code                   74 non-null     object 
 3   Sports Governing Body  74 non-null     object 
 4   1896                   74 non-null     object 
 5   1900                   74 non-null     object 
 6   1904                   74 non-null     object 
 7   1906*                  74 non-null     int64  
 8   1908                   74 non-null     int64  
 9   1912                   74 non-null     object 
 10  1920                   74 non-null     int64  
 11  1924                   74 non-null     object 
 12  1928                   72 non-null     object 
 13  1932                   72 non-null     object 
 14  1936                   72 non-null     object 
 15  1948    

In [14]:
print(pd.unique(program_data["Discipline"]))

['Artistic Swimming' 'Diving' 'Marathon Swimming' 'Swimming' 'Water Polo'
 'Archery' 'Athletics' 'Badminton' 'Baseball' 'Softball' '3x3'
 'Basketball' 'Basque Pelota' 'Boxing' 'Breaking' 'Sprint' 'Slalom'
 'Cricket' 'Croquet' 'BMX Freestyle' 'BMX Racing' 'Mountain Bike' 'Road'
 'Track' 'Dressage' 'Eventing' 'Jumping' 'Vaulting' 'Driving' 'Fencing'
 'Field hockey' 'Flag football' 'Football' 'Golf' 'Artistic' 'Rhythmic'
 'Trampoline' 'Indoor' 'Field' 'Jeu de Paume' 'Judo' 'Karate' 'Sixes' nan
 'Polo' 'Rackets' 'Roque' 'Coastal' 'Rowing' 'Sevens' 'Union' 'Sailing'
 'Shooting' 'Skateboarding' 'Sport Climbing' 'Squash' 'Surfing'
 'Table Tennis' 'Taekwondo' 'Tennis' 'Triathlon' 'Tug of War' 'Beach'
 'Weightlifting' 'Freestyle' 'Greco-Roman' 'Figure' 'Ice Hockey'
 'Total events' 'Total disciplines' 'Total sports']


In [15]:
# 清除不太可能存在于2028奥运会当中的项目
unlikely_events = [
    "Boxing", "Weightlifting", "Breaking",
    "Croquet", "Softball", "Lacrosse", 
    "Squash", "Cricket","Water Motorsports","Skating","Ice Hockey"
]

# Remove these unlikely events from the dataset
filtered_program_data = program_data[~program_data['Sport'].isin(unlikely_events)]
filtered_program_data.info()



<class 'pandas.core.frame.DataFrame'>
Index: 63 entries, 0 to 73
Data columns (total 35 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Sport                  63 non-null     object 
 1   Discipline             62 non-null     object 
 2   Code                   63 non-null     object 
 3   Sports Governing Body  63 non-null     object 
 4   1896                   63 non-null     object 
 5   1900                   63 non-null     object 
 6   1904                   63 non-null     object 
 7   1906*                  63 non-null     int64  
 8   1908                   63 non-null     int64  
 9   1912                   63 non-null     object 
 10  1920                   63 non-null     int64  
 11  1924                   63 non-null     object 
 12  1928                   63 non-null     object 
 13  1932                   63 non-null     object 
 14  1936                   63 non-null     object 
 15  1948         

1. **拳击（Boxing）**：由于国际拳击联合会的管理问题，拳击项目未被列入2028年洛杉矶奥运会的初始项目名单。尽管如此，如果相关问题得到解决，拳击仍有可能被重新纳入。 ([澎湃新闻](https://www.thepaper.cn/newsDetail_forward_15781574?utm_source=chatgpt.com))

2. **举重（Weightlifting）**：因反兴奋剂方面的历史问题，举重项目也未被列入初始名单。国际举重联合会需要进行改革，以确保该项目能够回归奥运会。 ([澎湃新闻](https://www.thepaper.cn/newsDetail_forward_15781574?utm_source=chatgpt.com))


1. **霹雳舞（Breaking）**：尽管在2024年巴黎奥运会上首次亮相，但霹雳舞未被纳入2028年洛杉矶奥运会的项目。 ([Diario AS](https://as.com/juegos_olimpicos/proxima-parada-hollywood-7000-millones-sin-coches-criquet-n/?utm_source=chatgpt.com))

2. **槌球（Croquet）**、**壘球（Softball）**、**长曲棍球（Lacrosse）**、**壁球（Squash）**、**板球（Cricket）**：这些项目在历史上曾出现在奥运会上，但在近期的奥运会中并未包含。然而，洛杉矶奥组委提议将棒垒球、板球、腰旗橄榄球、棍网球和壁球作为新增大项纳入2028年奥运会。 


In [16]:
# 填充缺失值
# 仅有Modern Pentathlon项目不存在归属，直接使用其名字作为discipline
filtered_program_data.fillna("Modern Pentathlon", inplace=True)

C:\Users\Tieti_Tage\AppData\Local\Temp\ipykernel_18968\1671211733.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_program_data.fillna("Modern Pentathlon", inplace=True)


In [17]:
# 转换类型
# 遍历所有列，将日期列的数据类型转换为整数
for col in filtered_program_data.columns:
    if col not in ["Discipline", "Sport", "Sports Governing Body","Code"]:
        # 将列数据转换为整数，跳过可能导致错误的数据
        filtered_program_data[col] = pd.to_numeric(filtered_program_data[col], errors='coerce').fillna(0).astype(int)

C:\Users\Tieti_Tage\AppData\Local\Temp\ipykernel_18968\3356668179.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_program_data[col] = pd.to_numeric(filtered_program_data[col], errors='coerce').fillna(0).astype(int)
C:\Users\Tieti_Tage\AppData\Local\Temp\ipykernel_18968\3356668179.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_program_data[col] = pd.to_numeric(filtered_program_data[col], errors='coerce').fillna(0).astype(int)
C:\Users\Tieti_Tage\AppData\Local\Temp\ipykernel_18968\3

In [18]:
filtered_program_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63 entries, 0 to 73
Data columns (total 35 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Sport                  63 non-null     object
 1   Discipline             63 non-null     object
 2   Code                   63 non-null     object
 3   Sports Governing Body  63 non-null     object
 4   1896                   63 non-null     int32 
 5   1900                   63 non-null     int32 
 6   1904                   63 non-null     int32 
 7   1906*                  63 non-null     int32 
 8   1908                   63 non-null     int32 
 9   1912                   63 non-null     int32 
 10  1920                   63 non-null     int32 
 11  1924                   63 non-null     int32 
 12  1928                   63 non-null     int32 
 13  1932                   63 non-null     int32 
 14  1936                   63 non-null     int32 
 15  1948                   63 non-

In [19]:
filtered_program_data.head(10)

,Sport,Discipline,Code,Sports Governing Body,1896,1900,1904,1906*,1908,1912,...,1988,1992,1996,2000,2004,2008,2012,2016,2020,2024
0,Aquatics,Artistic Swimming,SWA,World Aquatics,0,0,0,0,0,0,...,2,2,1,2,2,2,2,2,2,2
1,Aquatics,Diving,DIV,World Aquatics,0,0,2,1,2,4,...,4,4,4,8,8,8,8,8,8,8
2,Aquatics,Marathon Swimming,OWS,World Aquatics,0,0,0,0,0,0,...,0,0,0,0,0,2,2,2,2,2
3,Aquatics,Swimming,SWM,World Aquatics,4,7,9,4,6,9,...,31,31,32,32,32,32,32,32,35,35
4,Aquatics,Water Polo,WPO,World Aquatics,0,1,1,0,1,1,...,1,1,1,2,2,2,2,2,2,2
5,Archery,Archery,ARC,World Archery,0,7,6,0,3,0,...,4,4,4,4,4,4,4,4,5,5
6,Athletics,Athletics,ATH,World Athletics,12,23,25,21,26,30,...,42,43,44,46,46,47,47,47,48,48
7,Badminton,Badminton,BDM,BWF,0,0,0,0,0,0,...,0,4,5,5,5,5,5,5,5,5
8,Baseball and Softball,Baseball,BSB,WBSC[s1],0,0,0,0,0,0,...,0,1,1,1,1,1,0,0,1,0
9,Baseball and Softball,Softball,SBL,WBSC[s1],0,0,0,0,0,0,...,0,0,1,1,1,1,0,0,1,0


In [20]:
# 保存清洗后的数据
filtered_program_data.to_csv("2025_Problem_C_Data_cleaned\\filtered_program_data.csv", encoding="utf-8",index=False)


In [21]:
# # 只保留1950年以来的奥运会的项目情况
# # 删除第 4 到第 16 列
filtered_program_data.drop(columns=filtered_program_data.columns[4:16], inplace=True)
filtered_program_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63 entries, 0 to 73
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Sport                  63 non-null     object
 1   Discipline             63 non-null     object
 2   Code                   63 non-null     object
 3   Sports Governing Body  63 non-null     object
 4   1952                   63 non-null     int32 
 5   1956                   63 non-null     int32 
 6   1960                   63 non-null     int32 
 7   1964                   63 non-null     int32 
 8   1968                   63 non-null     int32 
 9   1972                   63 non-null     int32 
 10  1976                   63 non-null     int32 
 11  1980                   63 non-null     int32 
 12  1984                   63 non-null     int32 
 13  1988                   63 non-null     int32 
 14  1992                   63 non-null     int32 
 15  1996                   63 non-

C:\Users\Tieti_Tage\AppData\Local\Temp\ipykernel_18968\1440688301.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_program_data.drop(columns=filtered_program_data.columns[4:16], inplace=True)


In [22]:
rows_with_na = filtered_program_data[filtered_program_data.isna().any(axis=1)]
print("含缺失值的行：")
print(rows_with_na)

含缺失值的行：
Empty DataFrame
Columns: [Sport, Discipline, Code, Sports Governing Body, 1952, 1956, 1960, 1964, 1968, 1972, 1976, 1980, 1984, 1988, 1992, 1996, 2000, 2004, 2008, 2012, 2016, 2020, 2024]
Index: []

[0 rows x 23 columns]


# 读取东道主的数据

In [23]:
host_data = pd.read_csv("2025_Problem_C_Data\\summerOly_hosts.csv")

In [24]:
host_data

,Year,Host
0,1896,"Athens, Greece"
1,1900,"Paris, France"
2,1904,"St. Louis, United States"
3,1908,"London, United Kingdom"
4,1912,"Stockholm, Sweden"
5,1916,Cancelled (WWI – Berlin had been awarded)
6,1920,"Antwerp, Belgium"
7,1924,"Paris, France"
8,1928,"Amsterdam, Netherlands"
9,1932,"Los Angeles, United States"


# 读取并清洗运动员的数据

In [25]:
athletics_data = pd.read_csv("2025_Problem_C_Data\\summerOly_athletes.csv")

In [26]:
athletics_data

,Name,Sex,Team,NOC,Year,City,Sport,Event,Medal
0,A Dijiang,M,China,CHN,1992,Barcelona,Basketball,Basketball Men's Basketball,No medal
1,A Lamusi,M,China,CHN,2012,London,Judo,Judo Men's Extra-Lightweight,No medal
2,Gunnar Aaby,M,Denmark,DEN,1920,Antwerpen,Football,Football Men's Football,No medal
3,Edgar Aabye,M,Denmark/Sweden,DEN,1900,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,Cornelia (-strannood),F,Netherlands,NED,1932,Los Angeles,Athletics,Athletics Women's 100 metres,No medal
...,...,...,...,...,...,...,...,...,...
252560,Sefora Ada,F,Equatorial Guinea,GEQ,2024,Paris,Athletics,Women's 100m,No medal
252561,Emanuela Liuzzi,F,Italy,ITA,2024,Paris,Wrestling,Women's Freestyle 50kg,No medal
252562,Isayah Boers,M,Netherlands,NED,2024,Paris,Athletics,4 x 400m Relay Mixed,Gold
252563,Kevin Staut,M,France,FRA,2024,Paris,Equestrian,Jumping Team,Bronze


In [27]:
athletics_data.isna().sum()

Name     0
Sex      0
Team     0
NOC      0
Year     0
City     0
Sport    0
Event    0
Medal    0
dtype: int64

In [28]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Step 1: 加载数据
hosts = pd.read_csv('2025_Problem_C_Data/summerOly_hosts.csv', encoding='latin1')
# Step 2: 清洗数据
# 移除诡异的字符
hosts['Host'] = hosts['Host'].str.replace(r'[Â\xa0]', '', regex=True).str.strip()

In [1]:
import pandas as pd

# 1. 读取运动员数据
athletes = pd.read_csv('2025_Problem_C_Data/summerOly_athletes.csv', encoding='latin1')

# 2. 移除没有奖牌的数据
athletes = athletes[athletes['Medal'] != 'No medal']

# 3. 统计每个项目中不同奖牌的数量
medal_count = athletes.groupby(['Team', 'Year', 'Event', 'Medal']).size().reset_index(name='Medal_Count')

# 4. 判断是否为团体赛：如果某个项目中有多个运动员获得相同奖牌，则认为是团体赛
medal_count['Is_Team_Event'] = medal_count['Medal_Count'].apply(lambda x: 1 if x > 1 else 0)

# 5. 将团体赛的标记合并回运动员数据
athletes = pd.merge(athletes, medal_count[['Team', 'Year', 'Event', 'Medal', 'Is_Team_Event']], 
                    on=['Team', 'Year', 'Event', 'Medal'], how='left')

In [2]:
athletes.head()

,Name,Sex,Team,NOC,Year,City,Sport,Event,Medal,Is_Team_Event
0,Edgar Aabye,M,Denmark/Sweden,DEN,1900,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,1
1,Arvo Aaltonen,M,Finland,FIN,1920,Antwerpen,Swimming,Swimming Men's 200 metres Breaststroke,Bronze,0
2,Arvo Aaltonen,M,Finland,FIN,1920,Antwerpen,Swimming,Swimming Men's 400 metres Breaststroke,Bronze,0
3,Paavo Aaltonen,M,Finland,FIN,1948,London,Gymnastics,Gymnastics Men's Individual All-Around,Bronze,0
4,Paavo Aaltonen,M,Finland,FIN,1948,London,Gymnastics,Gymnastics Men's Team All-Around,Gold,1


In [4]:

# 6. 对团体赛进行处理：保留第一个获奖运动员，其他运动员的奖牌数设为0
# 按照 Team, Year, Event 和 Medal 排序，确保团体赛中的第一个运动员出现在前面
athletes = athletes.sort_values(by=['Team', 'Year', 'Event', 'Medal', 'Name'])

# 标记每个团体赛的第一个运动员
athletes['Is_First_Athlete'] = athletes.groupby(['Team', 'Year', 'Event', 'Medal'])['Name'].transform('first') == athletes['Name']
athletes.head()

,Name,Sex,Team,NOC,Year,City,Sport,Event,Medal,Is_Team_Event,Is_First_Athlete,Gold_Count,Silver_Count,Bronze_Count
7743,Jos Barrn,M,A North American Team,MEX,1900,Paris,Polo,Polo Men's Polo,Bronze,1,True,0,0,1
7744,Jos Barrn,M,A North American Team,MEX,1900,Paris,Polo,Polo Men's Polo,Bronze,1,True,0,0,1
7745,Jos Barrn,M,A North American Team,MEX,1900,Paris,Polo,Polo Men's Polo,Bronze,1,True,0,0,1
33098,William Wright,M,A North American Team,USA,1900,Paris,Polo,Polo Men's Polo,Bronze,1,False,0,0,0
37901,Ivan Litvinovich,M,AIN,AIN,2024,Paris,Trampoline Gymnastics,Men,Gold,0,True,1,0,0


In [3]:

# 7. 计算奖牌数：对于团体赛，只有第一个运动员的奖牌数为1，其他为0
athletes['Gold_Count'] = athletes.apply(lambda row: 1 if row['Medal'] == 'Gold' and row['Is_First_Athlete'] else 0, axis=1)
athletes['Silver_Count'] = athletes.apply(lambda row: 1 if row['Medal'] == 'Silver' and row['Is_First_Athlete'] else 0, axis=1)
athletes['Bronze_Count'] = athletes.apply(lambda row: 1 if row['Medal'] == 'Bronze' and row['Is_First_Athlete'] else 0, axis=1)

# 8. 获取 Event 对应的 Sport 信息
# 假设 athletes 数据中有 Sport 列，将该列加入到 medal_count
medal_count = pd.merge(medal_count, athletes[['Event', 'Sport']].drop_duplicates(), on='Event', how='left')

# 9. 按照国家、年份、项目、奖牌类型汇总奖牌数量
medal_summary = athletes.groupby(['Team', 'Year', 'Sport']).agg({
    'Gold_Count': 'sum',
    'Silver_Count': 'sum',
    'Bronze_Count': 'sum'
}).reset_index()

# 10. 去除 Event 列（如果存在的话）
medal_summary = medal_summary.drop(columns=['Event'], errors='ignore')

# 输出汇总结果
medal_summary.head()

,Team,Year,Sport,Gold_Count,Silver_Count,Bronze_Count
0,A North American Team,1900,Polo,0,0,3
1,AIN,2024,Rowing,0,1,0
2,AIN,2024,Tennis,0,1,0
3,AIN,2024,Trampoline Gymnastics,1,1,0
4,AIN,2024,Weightlifting,0,0,1


In [32]:
pivot_table = medal_summary.copy()
pivot_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7018 entries, 0 to 7017
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Team          7018 non-null   object
 1   Year          7018 non-null   int64 
 2   Sport         7018 non-null   object
 3   Gold_Count    7018 non-null   int64 
 4   Silver_Count  7018 non-null   int64 
 5   Bronze_Count  7018 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 329.1+ KB


In [33]:
print(pivot_table.head(4))

                    Team  Year                  Sport  Gold_Count  \
0  A North American Team  1900                   Polo           0   
1                    AIN  2024                 Rowing           0   
2                    AIN  2024                 Tennis           0   
3                    AIN  2024  Trampoline Gymnastics           1   

   Silver_Count  Bronze_Count  
0             0             3  
1             1             0  
2             1             0  
3             1             0  


In [34]:
# 删除1950前的数据
pivot_table = pivot_table[pivot_table["Year"]>1950]

In [35]:
pivot_table.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5582 entries, 1 to 7016
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Team          5582 non-null   object
 1   Year          5582 non-null   int64 
 2   Sport         5582 non-null   object
 3   Gold_Count    5582 non-null   int64 
 4   Silver_Count  5582 non-null   int64 
 5   Bronze_Count  5582 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 305.3+ KB


In [36]:
# 标准化国家名称
pivot_table['Team'] = pivot_table['Team'].str.replace(r'-\d+$', '', regex=True)


In [37]:
# 保存表格
output_path = '2025_Problem_C_Data_cleaned/olympic_medal_analysis.csv'
pivot_table.to_csv(output_path, index=False)

# 合并表格数据,将cleaned_medal_data当中的数据与运动项目的数据进行合并，同时考虑东道主效应的影响

In [38]:
import pandas as pd

# ======================================================================
# 第一阶段：国家名称标准化处理（核心逻辑）
# ======================================================================
# 定义需移除的国家列表和名称映射规则
final_remove_nocs = [
    "Bohemia", "Australasia", "Russian Empire", "Czechoslovakia",
    "Unified Team", "Mixed team", "East Germany", "West Germany",
    "United Team of Germany", "FR Yugoslavia", "Serbia and Montenegro",
    "Soviet Union"
]

final_merge_nocs = {
    "Formosa": "Chinese Taipei",
    "Hong Kong": "Hong Kong, China",
    "AIN": "Russia"
}

def standardize_noc_data(df, noc_col='NOC'):
    """执行国家名称标准化处理"""
    # 移除无效国家
    df = df[~df[noc_col].isin(final_remove_nocs)]
    
    # 合并国家名称
    df[noc_col] = df[noc_col].replace(final_merge_nocs)
    
    # 清洗特殊字符
    df[noc_col] = df[noc_col].str.replace(r'-\d+$', '', regex=True)
    
    return df

# ======================================================================
# 第二阶段：加载并处理原始数据
# ======================================================================
# 加载原始数据
file_path_original = '2025_Problem_C_Data_cleaned/olympic_medal_analysis.csv'
data = pd.read_csv(file_path_original)

# 执行标准化处理
data = standardize_noc_data(data.rename(columns={'Team': 'NOC'}))

# 重命名列并添加合计列
data.rename(columns={
    "Silver_Count": "Silver",
    "Gold_Count": "Gold",
    "Bronze_Count": "Bronze"
}, inplace=True)

if 'Total' not in data.columns:
    data['Total'] = data[['Gold', 'Silver', 'Bronze']].sum(axis=1)

# ======================================================================
# 第三阶段：东道主标记处理
# ======================================================================
year_host_mapping = {
    1992: "Spain",
    1996: "United States",
    2000: "Australia",
    2004: "Greece",
    2008: "China",
    2012: "Great Britain",
    2016: "Brazil",
    2020: "Japan",
    2024: "France"
}

# 确保映射中的国家名称已标准化
year_host_mapping = {k: standardize_noc_data(pd.DataFrame({'NOC': [v]})).iloc[0]['NOC'] 
                    for k, v in year_host_mapping.items()}

data['Is_Host'] = data.apply(
    lambda row: 1 if row['Year'] in year_host_mapping and 
    row['NOC'] == year_host_mapping[row['Year']] else 0, 
    axis=1
)

# ======================================================================
# 第四阶段：创建项目奖牌数据透视表
# ======================================================================
# 定义需要排除的非奥运项目
unlikely_events = ['Aeronautics', 'Alpinism', 'Basque pelota']

def create_medal_pivot(df, medal_type):
    """创建指定奖牌类型的数据透视表"""
    pivot = df.pivot_table(
        index=['NOC', 'Year', 'Is_Host'],
        columns='Sport',
        values=medal_type,
        aggfunc='sum',
        fill_value=0
    )
    
    # 移除非常规项目
    existing_cols = pivot.columns.intersection(unlikely_events)
    if not existing_cols.empty:
        pivot = pivot.drop(columns=existing_cols)
    
    # 合并汇总数据
    summary = df.groupby(['NOC', 'Year', 'Is_Host'])[[medal_type]].sum()
    return pd.concat([summary, pivot], axis=1).reset_index()

# 生成各奖牌类型数据
gold_data = create_medal_pivot(data, 'Gold')
silver_data = create_medal_pivot(data, 'Silver')
bronze_data = create_medal_pivot(data, 'Bronze')

# ======================================================================
# 第五阶段：保存处理结果
# ======================================================================
# 定义保存路径
save_path = '2025_Problem_C_Data_cleaned/'
gold_data.to_csv(f'{save_path}gold_medal_data.csv', index=False)
silver_data.to_csv(f'{save_path}silver_medal_data.csv', index=False)
bronze_data.to_csv(f'{save_path}bronze_medal_data.csv', index=False)

# 输出验证信息
print(f"处理完成，共清洗{len(final_remove_nocs)}个无效国家")
print(f"国家名称映射应用情况：{final_merge_nocs}")
print(f"各数据集维度：")
print(f"金牌数据：{gold_data.shape}")
print(f"银牌数据：{silver_data.shape}")
print(f"铜牌数据：{bronze_data.shape}")

处理完成，共清洗12个无效国家
国家名称映射应用情况：{'Formosa': 'Chinese Taipei', 'Hong Kong': 'Hong Kong, China', 'AIN': 'Russia'}
各数据集维度：
金牌数据：(1191, 59)
银牌数据：(1191, 59)
铜牌数据：(1191, 59)


In [39]:
# import pandas as pd
# 
# # 加载数据文件
# file_path_original = '2025_Problem_C_Data_cleaned/olympic_medal_analysis.csv'
# file_path_new = '2025_Problem_C_Data_cleaned/cleaned_medal_data.csv'
# 
# data = pd.read_csv(file_path_original)
# new_data = pd.read_csv(file_path_new)
# 
# # 手动创建 1992 年以来的东道主国家清洁数据
# year_host_mapping = {
#     1992: "Spain",
#     1996: "United States",
#     2000: "Australia",
#     2004: "Greece",
#     2008: "China",
#     2012: "Great Britain",
#     2016: "Brazil",
#     2020: "Japan",
#     2024: "France"
# }
# # 标准化国家名称，移除例如 '-1' 的后缀
# data['Team'] = data['Team'].str.replace(r'-\d+$', '', regex=True)
# data.rename(columns={'Team': 'NOC',"Silver_Count":"Silver",
#                      "Gold_Count":"Gold","Bronze_Count":"Bronze"}, inplace=True)
# 
# # 添加东道主效应
# # 标记每个国家是否是该年的东道主
# data['Is_Host'] = data.apply(lambda row: 1 if row['Year'] in year_host_mapping and row['NOC'] == year_host_mapping[row['Year']] else 0, axis=1)
# 
# # 步骤 1: 数据清洗
# # 确保存在 'Total' 列，如果没有则通过求和 'Gold', 'Silver', 'Bronze' 列创建
# if 'Total' not in data.columns:
#     data['Total'] = data[['Gold', 'Silver', 'Bronze']].sum(axis=1)
# 
# # 步骤 2: 创建按国家、年份和运动项目统计的奖牌数据透视表
# # 创建金牌数据表
# gold_pivot = data.pivot_table(
#     index=['NOC', 'Year','Is_Host'], 
#     columns='Sport', 
#     values='Gold', 
#     aggfunc='sum', 
#     fill_value=0
# )
# gold_pivot = gold_pivot.drop(columns=unlikely_events, errors='ignore')  # 移除不可能的项目
# gold_summary = data.groupby(['NOC', 'Year','Is_Host'])[['Gold']].sum()
# gold_data = pd.concat([gold_summary, gold_pivot], axis=1).reset_index()
# gold_file_path = '2025_Problem_C_Data_cleaned/gold_medal_data.csv'
# gold_data.to_csv(gold_file_path, index=False)
# 
# # 创建银牌数据表
# silver_pivot = data.pivot_table(
#     index=['NOC', 'Year','Is_Host'], 
#     columns='Sport', 
#     values='Silver', 
#     aggfunc='sum', 
#     fill_value=0
# )
# silver_pivot = silver_pivot.drop(columns=unlikely_events, errors='ignore')
# silver_summary = data.groupby(['NOC','Year','Is_Host'])[['Silver']].sum()
# silver_data = pd.concat([silver_summary, silver_pivot], axis=1).reset_index()
# silver_file_path = '2025_Problem_C_Data_cleaned/silver_medal_data.csv'
# silver_data.to_csv(silver_file_path, index=False)
# 
# # 创建铜牌数据表
# bronze_pivot = data.pivot_table(
#     index=['NOC', 'Year','Is_Host'], 
#     columns='Sport', 
#     values='Bronze', 
#     aggfunc='sum', 
#     fill_value=0
# )
# 
# bronze_pivot = bronze_pivot.drop(columns=unlikely_events, errors='ignore')
# bronze_summary = data.groupby(['NOC', 'Year','Is_Host'])[['Bronze']].sum()
# bronze_data = pd.concat([bronze_summary, bronze_pivot], axis=1).reset_index()
# bronze_file_path = '2025_Problem_C_Data_cleaned/bronze_medal_data.csv'
# bronze_data.to_csv(bronze_file_path, index=False)
# 
# # 输出保存的文件路径
# print("Gold medals saved to:", gold_file_path)
# print("Silver medals saved to:", silver_file_path)
# print("Bronze medals saved to:", bronze_file_path)

# 建立模型

In [40]:
# 加载数据
file_path_gold = '2025_Problem_C_Data_cleaned/gold_medal_data.csv'
file_path_silver = '2025_Problem_C_Data_cleaned/silver_medal_data.csv'
file_path_bronze = '2025_Problem_C_Data_cleaned/bronze_medal_data.csv'

gold_data = pd.read_csv(file_path_gold)
silver_data = pd.read_csv(file_path_silver)
bronze_data = pd.read_csv(file_path_bronze)

# 检查数据结构
gold_data.head()

,NOC,Year,Is_Host,Gold,3x3 Basketball,Archery,Artistic Gymnastics,Artistic Swimming,Athletics,Badminton,...,Table Tennis,Taekwondo,Tennis,Trampoline Gymnastics,Trampolining,Triathlon,Volleyball,Water Polo,Weightlifting,Wrestling
0,Afghanistan,2008,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Afghanistan,2012,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Albania,2024,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Algeria,1984,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Algeria,1992,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
silver_data.head()

,NOC,Year,Is_Host,Silver,3x3 Basketball,Archery,Artistic Gymnastics,Artistic Swimming,Athletics,Badminton,...,Table Tennis,Taekwondo,Tennis,Trampoline Gymnastics,Trampolining,Triathlon,Volleyball,Water Polo,Weightlifting,Wrestling
0,Afghanistan,2008,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Afghanistan,2012,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Albania,2024,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Algeria,1984,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Algeria,1992,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
bronze_data.head()

,NOC,Year,Is_Host,Bronze,3x3 Basketball,Archery,Artistic Gymnastics,Artistic Swimming,Athletics,Badminton,...,Table Tennis,Taekwondo,Tennis,Trampoline Gymnastics,Trampolining,Triathlon,Volleyball,Water Polo,Weightlifting,Wrestling
0,Afghanistan,2008,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,Afghanistan,2012,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,Albania,2024,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,Algeria,1984,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Algeria,1992,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:


from sklearn.metrics import mean_squared_error


def optimized_complete_years(data, start_year=1992, end_year=2024):
    """
    Optimized version of the function to handle the 'Is_Host' column, without filling missing years.

    Parameters:
        data (pd.DataFrame): Input data with columns 'NOC', 'Year', and other metrics.
        start_year (int): Starting year for the Olympics.
        end_year (int): Ending year for the Olympics.

    Returns:
        pd.DataFrame: Data with the 'Is_Host' column properly handled.
    """
    # Ensure Is_Host is appropriately handled (default to 0 for missing years)
    if 'Is_Host' in data.columns:
        data['Is_Host'] = data['Is_Host'].fillna(0).astype(int)
    else:
        data['Is_Host'] = 0

    return data

In [91]:
import pandas as pd
from joblib import Parallel, delayed
from scipy.stats import poisson
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from scipy.stats import uniform
from tqdm import tqdm

# 列名处理（统一转换所有列名）
def safe_column_name(s):
    return s.strip().replace("/", "_").replace(" ", "_").replace("-", "_")

def preprocess_columns(df):
    """预处理所有列名"""
    df = df.rename(columns=lambda x: safe_column_name(x))
    return df

def build_historical_features(group):
    """动态构建时序特征（修复列名问题）"""
    features = []
    group = preprocess_columns(group)  # 确保列名统一
    event_cols = [c for c in group.columns if c not in ['NOC', 'Year', 'Is_Host']]
    group = group.sort_values('Year')
    
    for idx, row in group.iterrows():
        current_year = row['Year']
        for event in event_cols:
            # 获取原始事件名称对应的数据列
            raw_event = event  # 已转换后的列名
            history = group[(group['Year'] < current_year) & 
                           (group[raw_event].notna())]
            
            # 生成特征名称（无需再次转换）
            stats = {
                f'{event}_median': history[raw_event].median(skipna=True),
                f'{event}_std': history[raw_event].std(skipna=True),
                f'{event}_trend': history[raw_event].diff().mean(skipna=True)
            }
            
            features.append({
                'NOC': row['NOC'],
                'Year': current_year,
                'Is_Host': row['Is_Host'],
                'Event': event,  # 存储转换后的列名
                **stats,
                'Target': row[raw_event]
            })
    
    return pd.DataFrame(features)

def train_single_model(noc, event, group):
    """训练单个国家-项目组合模型（修复列名问题）"""
    try:
        # 统一列名格式
        group = preprocess_columns(group)
        event = safe_column_name(event)
        
        # 特征工程
        feature_df = build_historical_features(group)
        
        if len(feature_df) < 2:
            return default_prediction(noc, event, group)
        
        # 准备数据（使用转换后的列名）
        X = feature_df[[ 
            'Year', 'Is_Host',
            f'{event}_median',
            f'{event}_std',
            f'{event}_trend'
        ]]
        y = feature_df['Target']
        
        tscv = TimeSeriesSplit(n_splits=min(4, len(X)-1))
        
        # 超参数搜索
        param_dist = {
            'n_estimators': [30, 50, 100],
            'max_depth': [2, 3],
            'learning_rate': uniform(0.005,0.01),
            'subsample': uniform(0.5, 0.5)
        }
        
        # 模型训练
        model = RandomizedSearchCV(
            GradientBoostingRegressor(),
            param_dist,
            n_iter=5,
            cv=tscv,
            n_jobs=1
        )
        model.fit(X, y)
        
        # 构建2028年特征
        latest = build_2028_features(noc, event, group)
        
        # 执行预测
        pred = model.predict(latest)[0]  # pred在此定义
        
        # 使用泊松分布计算置信区间
        # 计算置信区间，使用95%的置信区间（1 - α = 0.95），α = 0.05
        lower = poisson.ppf(0.025, pred)  # 2.5%分位数
        upper = poisson.ppf(0.975, pred)  # 97.5%分位数
        
        return {
            'NOC': noc,
            'Event': event,
            'Predicted': max(0, pred),
            'Lower': max(0, lower),
            'Upper': max(0, upper)
        }
        
    except Exception as e:
        print(f"Error in {noc}-{event}: {str(e)}")
        return default_prediction(noc, event, group)

def build_2028_features(noc, event, group):
    """构建2028年预测特征（修复列名问题）"""
    group = preprocess_columns(group)
    event = safe_column_name(event)
    history = group[group['Year'] < 2028]
    is_host = 1 if 'United_States' in group['NOC'].values else 0  # 注意NOC也被转换
    
    return pd.DataFrame([{
        'Year': 2028,
        'Is_Host': is_host,
        f'{event}_median': history[event].median(skipna=True),
        f'{event}_std': history[event].std(skipna=True),
        f'{event}_trend': history[event].diff().mean(skipna=True)
    }])

def default_prediction(noc, event, group):
    """默认预测策略（修复列名问题）"""
    group = preprocess_columns(group)
    event = safe_column_name(event)
    last = group[event].iloc[-1] if len(group) > 0 else 0
    return {
        'NOC': safe_column_name(noc),  # NOC也统一格式
        'Event': event,
        'Predicted': max(0, last),
        'Lower': max(0, last - 1),
        'Upper': max(0, last + 1)
    }

def predict_2028(data):
    """主预测流程（增加列名预处理）"""
    # 预处理输入数据列名
    data = preprocess_columns(data.copy())
    
    # 获取所有项目列（使用转换后的列名）
    non_event_cols = ['NOC', 'Year', 'Is_Host']
    event_cols = [c for c in data.columns if c not in non_event_cols]
    
    # 并行处理
    results = Parallel(n_jobs=-1, verbose=1)(
        delayed(train_single_model)(
            noc=noc,
            event=event,
            group=data[data['NOC'] == noc][non_event_cols + [event]].copy()
        )
        for noc in tqdm(data['NOC'].unique(), desc='Processing Countries')
        for event in event_cols
    )
    
    # 处理结果
    df = pd.DataFrame(results)
    df['Predicted'] = df['Predicted'].round().astype(int)
    df['Lower'] = df['Lower'].round().astype(int)
    df['Upper'] = df['Upper'].round().astype(int)
    
    return df

In [92]:

# 数据预处理
bronze_data = optimized_complete_years(bronze_data)
gold_data = optimized_complete_years(gold_data)
silver_data = optimized_complete_years(silver_data)

# 将填补的年份数据保存为 CSV 文件
bronze_data.to_csv('bronze_data.csv', index=False)
gold_data.to_csv('gold_data.csv', index=False)
silver_data.to_csv('silver_data.csv', index=False)

In [89]:
gold_data.head()


,NOC,Year,Is_Host,Gold,3x3 Basketball,Archery,Artistic Gymnastics,Artistic Swimming,Athletics,Badminton,...,Table Tennis,Taekwondo,Tennis,Trampoline Gymnastics,Trampolining,Triathlon,Volleyball,Water Polo,Weightlifting,Wrestling
0,Afghanistan,2008,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Afghanistan,2012,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Albania,2024,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Algeria,1984,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Algeria,1992,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [90]:
gold_predictions = predict_2028(gold_data)

Processing Countries: 100%|██████████| 210/210 [00:41<00:00,  5.10it/s]
[Parallel(n_jobs=-1)]: Done 11760 out of 11760 | elapsed:   42.1s finished


In [84]:
gold_predictions.head()

,NOC,Event,Predicted,Lower,Upper
0,Afghanistan,Gold,0,0,1
1,Afghanistan,3x3_Basketball,0,0,1
2,Afghanistan,Archery,0,0,1
3,Afghanistan,Artistic_Gymnastics,0,0,1
4,Afghanistan,Artistic_Swimming,0,0,1


In [93]:
gold_predictions.to_csv('gold_predictions_2028.csv', index=False)

In [86]:

silver_predictions = predict_2028(silver_data)
silver_predictions.to_csv('silver_predictions_2028.csv', index=False)

Processing Countries: 100%|██████████| 210/210 [00:43<00:00,  4.87it/s]
[Parallel(n_jobs=-1)]: Done 11760 out of 11760 | elapsed:   44.4s finished


In [51]:

bronze_predictions = predict_2028(bronze_data)
bronze_predictions.to_csv('bronze_predictions_2028.csv', index=False)


Processing Countries: 100%|██████████| 210/210 [00:46<00:00,  4.52it/s]
[Parallel(n_jobs=-1)]: Done 11760 out of 11760 | elapsed:   47.2s finished


- 预测结果保留在表中

# 伟大教练效应分析

In [96]:
df = pd.read_csv('2025_Problem_C_Data/summerOly_athletes.csv', encoding='latin1')


- 通过检测明显的奖牌获得的波动，来判断是否有伟大教练效应的出现。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from statsmodels.tsa.statespace.tools import cusum_squares
from sklearn.preprocessing import MinMaxScaler

# ====================
# 数据预处理与奖牌编码
# ====================
def load_and_preprocess(filepath):
    """加载数据并进行预处理"""
    df = pd.read_csv(filepath)
    
    # 奖牌数值映射（保留等级信息）
    medal_map = {
        'No medal': 0,
        'Bronze': 1,
        'Silver': 2,
        'Gold': 3
    }
    df['Medal_Value'] = df['Medal'].map(medal_map)
    
    # 生成二值化特征
    df['Has_Medal'] = np.where(df['Medal'] != 'No medal', 1, 0)
    
    return df

# ====================
# 特征工程模块
# ====================
def calculate_features(df):
    """计算关键特征"""
    
    # 按国家-项目-年份聚合奖牌数据
    medal_df = df.groupby(['NOC','Sport','Year'])['Medal_Value'].sum().reset_index()
    
    # 运动员流动性计算（Jaccard相似性）
    athlete_sets = df.groupby(['NOC','Sport','Year'])['Name'].apply(set)
    jaccard = athlete_sets.groupby(['NOC','Sport']).apply(
        lambda x: len(x.iloc[-1] & x.iloc[-2])/len(x.iloc[-1] | x.iloc[-2]) 
        if len(x) >=2 else 0
    ).reset_index(name='Athlete_Jaccard')
    
    # 赫芬达尔指数计算
    event_hhi = df[df['Medal_Value']>0].groupby(
        ['NOC','Sport','Year']
    )['Event'].apply(
        lambda x: (x.value_counts(normalize=True)**2).sum()
    ).reset_index(name='HHI')
    
    # 合并特征
    features = pd.merge(
        medal_df,
        jaccard,
        on=['NOC','Sport'],
        how='left'
    )
    features = pd.merge(
        features,
        event_hhi,
        on=['NOC','Sport','Year'],
        how='left'
    )
    
    # 特征填充与平滑
    features['HHI'] = features.groupby(['NOC','Sport'])['HHI'].ffill().fillna(0)
    features['Athlete_Jaccard'] = features['Athlete_Jaccard'].fillna(0)
    
    return features

# ====================
# 可视化模块
# ====================
def plot_coach_effect_triangulation(features, noc, sport):
    """综合可视化教练效应三要素"""
    
    target_data = features[
        (features['NOC']==noc) & 
        (features['Sport']==sport)
    ].set_index('Year')
    
    # 创建画布
    fig = plt.figure(figsize=(15,10))
    gs = fig.add_gridspec(3, 2)
    
    # 子图1：奖牌趋势与CUSUM
    ax1 = fig.add_subplot(gs[0, :])
    ax1.plot(target_data.index, target_data['Medal_Value'], 
            marker='o', color='#1f77b4', label='奖牌总分')
    ax1.set_title(f"{noc} {sport} 奖牌趋势分析", fontsize=14, pad=20)
    
    # 添加CUSUM控制线
    cusum_values = cusum_squares(target_data['Medal_Value'] - target_data['Medal_Value'].mean())
    ax1.plot(target_data.index, cusum_values, 
            linestyle='--', color='#ff7f0e', label='CUSUM统计量')
    ax1.axhline(5, color='#2ca02c', linestyle=':', label='显著性阈值')
    ax1.legend()
    
    # 子图2：运动员流动桑基图
    ax2 = fig.add_subplot(gs[1, 0])
    athlete_retention = target_data['Athlete_Jaccard'].values
    ax2.plot(target_data.index, athlete_retention, 
            marker='s', color='#d62728', linewidth=2)
    ax2.set_ylim(0, 1)
    ax2.set_title("运动员保留率 (Jaccard指数)", fontsize=12)
    ax2.fill_between(target_data.index, 0, athlete_retention, 
                    color='#d62728', alpha=0.1)
    
    # 子图3：项目集中度分析
    ax3 = fig.add_subplot(gs[1, 1])
    hhi_values = target_data['HHI'].values
    ax3.plot(target_data.index, hhi_values, 
            marker='^', color='#9467bd', linewidth=2)
    ax3.set_ylim(0, 1)
    ax3.set_title("项目集中度 (HHI指数)", fontsize=12)
    ax3.fill_between(target_data.index, hhi_values, 0, 
                    color='#9467bd', alpha=0.1)
    
    # 子图4：三维特征空间
    ax4 = fig.add_subplot(gs[2, :], projection='3d')
    ax4.scatter(
        cusum_values, 
        1 - athlete_retention, 
        hhi_values,
        c=target_data.index,
        cmap='viridis',
        s=50
    )
    ax4.set_xlabel("CUSUM统计量", labelpad=10)
    ax4.set_ylabel("运动员更新率", labelpad=10)
    ax4.set_zlabel("集中度变化", labelpad=10)
    ax4.set_title("三维特征空间分布", y=1.05, fontsize=12)
    
    plt.tight_layout()
    plt.show()

def generate_interactive_sankey(df, noc, sport, year):
    """生成交互式桑基图"""
    
    # 获取前后两届运动员名单
    prev_year = year - 4
    athletes = {
        'prev': set(df[(df.NOC==noc) & (df.Sport==sport) & (df.Year==prev_year)]['Name']),
        'current': set(df[(df.NOC==noc) & (df.Sport==sport) & (df.Year==year)]['Name'])
    }
    
    # 计算节点关系
    retained = athletes['prev'] & athletes['current']
    new_athletes = athletes['current'] - athletes['prev']
    
    # 构造桑基图数据
    labels = ['上一届', '保留队员', '新队员']
    source = [0, 0]
    target = [1, 2]
    value = [len(retained), len(new_athletes)]
    
    # 可视化
    fig = go.Figure(go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=labels,
            color=["#8c564b", "#2ca02c", "#17becf"]
        ),
        link=dict(
            source=source,
            target=target,
            value=value,
            color=["rgba(140,86,75,0.3)", "rgba(23,190,207,0.3)"]
        )
    ))
    
    fig.update_layout(
        title=f"{noc} {sport} {year}年运动员流动分析",
        font_size=12,
        width=800,
        height=500
    )
    fig.show()

# ====================
# 统计检测模块
# ====================
class CoachEffectDetector:
    """教练效应检测器"""
    
    def __init__(self, cusum_threshold=5, jaccard_threshold=0.3, hhi_threshold=0.2):
        self.cusum_threshold = cusum_threshold
        self.jaccard_threshold = jaccard_threshold
        self.hhi_threshold = hhi_threshold
        
    def detect(self, features, noc, sport):
        """检测指定国家/项目的教练效应"""
        target = features[
            (features['NOC']==noc) & 
            (features['Sport']==sport)
        ].set_index('Year')
        
        if len(target) < 3:
            return False  # 数据不足
        
        # 计算三信号
        cusum = cusum_squares(target['Medal_Value']).iloc[-1]
        jaccard = target['Athlete_Jaccard'].iloc[-1]
        hhi_change = target['HHI'].iloc[-1] - target['HHI'].mean()
        
        return (cusum > self.cusum_threshold) & \
               (jaccard < self.jaccard_threshold) & \
               (hhi_change > self.hhi_threshold)

# ====================
# 主程序
# ====================
if __name__ == "__main__":
    # 数据加载与预处理
    df = load_and_preprocess("athlete_events.csv")
    
    # 特征计算
    features = calculate_features(df)
    
    # 示例分析：芬兰体操1948
    print("\n=== 芬兰体操案例可视化 ===")
    plot_coach_effect_triangulation(features, 'FIN', 'Gymnastics')
    generate_interactive_sankey(df, 'FIN', 'Gymnastics', 1948)
    
    # 自动检测潜在案例
    print("\n=== 潜在教练效应检测 ===")
    detector = CoachEffectDetector()
    candidates = features.groupby(['NOC','Sport']).apply(
        lambda x: detector.detect(features, x.name[0], x.name[1])
    )
    positive_cases = candidates[candidates].index.tolist()
    
    if positive_cases:
        print(f"检测到{len(positive_cases)}个潜在案例：")
        for i, (noc, sport) in enumerate(positive_cases, 1):
            print(f"{i}. {noc} - {sport}")
    else:
        print("未检测到显著案例")